# Ridge and Lasso Regression - Lab

## Introduction

In this lab, you'll practice your knowledge on Ridge and Lasso regression!

## Objectives

You will be able to:

- Use Lasso and ridge regression in Python
- Compare Lasso and Ridge with standard regression

## Housing Prices Data

Let's look at yet another house pricing data set.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('Housing_Prices/train.csv')

Look at df.info

In [2]:
# Your code here
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

We'll make a first selection of the data by removing some of the data with `dtype = object`, this way our first model only contains **continuous features**

Make sure to remove the SalesPrice column from the predictors (which you store in `X`), then replace missing inputs by the median per feature.

Store the target in `y`.

In [3]:
# Load necessary packages
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# remove "object"-type features and SalesPrice from `X`
features = [col for col in df.columns if df[col].dtype != object and col != 'SalePrice']
X = df[features]

# Impute null values
for col in X:
    med = X[col].median()
    X[col].fillna(value = med, inplace = True)

# Create y
y = df.SalePrice

Look at the information of `X` again

In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 37 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null int64
BsmtFinSF2       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
1stFlrSF         1460 non-null int64
2ndFlrSF         1460 non-null int64
LowQualFinSF     1460 non-null int64
GrLivArea        1460 non-null int64
BsmtFullBath     1460 non-null int64
BsmtHalfBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
F

## Let's use this data to perform a first naive linear regression model

Compute the R squared and the MSE for both train and test set.

In [5]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error

# Split in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Fit the model and print R2 and MSE for train and test
linreg = LinearRegression()
linreg.fit(X_train, y_train)

print('Train R2: ', linreg.score(X_train, y_train))
print('Test R2: ', linreg.score(X_test, y_test))
print('Train MSE: ', mean_squared_error(y_train, linreg.predict(X_train)))
print('Test MSE: ', mean_squared_error(y_test, linreg.predict(X_test)))

Train R2:  0.8105448730742599
Test R2:  0.8050349892964999
Train MSE:  1282546281.7309353
Test MSE:  933743938.7460202


## Normalize your data

We haven't normalized our data, let's create a new model that uses `preprocessing.scale` to scale our predictors!

In [6]:
from sklearn import preprocessing

# scale the data and perform train test split
X_scaled = preprocessing.scale(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled,y)

Perform the same linear regression on this data and print out R-squared and MSE.

In [7]:
# Your code here
linreg_norm = LinearRegression()
linreg_norm.fit(X_train, y_train)

print('Train R2: ', linreg.score(X_train, y_train))
print('Test R2: ', linreg.score(X_test, y_test))
print('Train MSE: ', mean_squared_error(y_train, linreg.predict(X_train)))
print('Test MSE: ', mean_squared_error(y_test, linreg.predict(X_test)))

Train R2:  -50.536109946222325
Test R2:  -53.95832732987259
Train MSE:  330276964365.2727
Test MSE:  328844426919.38116


## Include dummy variables

We haven't included dummy variables so far: let's use our "object" variables again and create dummies

In [8]:
# Create X_cat which contains only the categorical variables
features_obj = [col for col in df.columns if df[col].dtypes == object]
X_cat = df[features_obj]

In [9]:
X_cat.shape

(1460, 43)

In [10]:
# Make dummies
X_cat = pd.get_dummies(X_cat)
X_cat.shape

(1460, 252)

Merge `x_cat` together with our scaled `X` so you have one big predictor dataframe.

In [11]:
# Your code here
X_all = pd.concat([pd.DataFrame(X_scaled), X_cat], axis=1)

In [12]:
X_all.head()

,0,1,2,3,4,5,6,7,8,9,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-1.730865,0.073375,-0.220875,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.514104,0.575425,...,0,0,0,1,0,0,0,0,1,0
1,-1.728492,-0.872563,0.460320,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.570750,1.171992,...,0,0,0,1,0,0,0,0,1,0
2,-1.726120,0.073375,-0.084636,0.073480,0.651479,-0.517200,0.984752,0.830215,0.325915,0.092907,...,0,0,0,1,0,0,0,0,1,0
3,-1.723747,0.309859,-0.447940,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.570750,-0.499274,...,0,0,0,1,1,0,0,0,0,0
4,-1.721374,0.073375,0.641972,0.375148,1.374795,-0.517200,0.951632,0.733308,1.366489,0.463568,...,0,0,0,1,0,0,0,0,1,0


Perform the same linear regression on this data and print out R-squared and MSE.

In [13]:
# Your code here
# Split in train and test
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X_all, y)

# Fit the model and print R2 and MSE for train and test
linreg_all = LinearRegression()
linreg_all.fit(X_train_all, y_train_all)

print('Train R2: ', linreg_all.score(X_train_all, y_train_all))
print('Test R2: ', linreg_all.score(X_test_all, y_test_all))
print('Train MSE: ', mean_squared_error(y_train, linreg_all.predict(X_train_all)))
print('Test MSE: ', mean_squared_error(y_test, linreg_all.predict(X_test_all)))

Train R2:  0.9381918954291198
Test R2:  -7.5027603492906e+19
Train MSE:  12668158818.17717
Test MSE:  4.75400010351012e+29


Notice the severe overfitting above; our training R squared is quite high, but the testing R squared is negative! Our predictions are far far off. Similarly, the scale of the Testing MSE is orders of magnitude higher then that of the training.

## Perform Ridge and Lasso regression

Use all the data (normalized features and dummy categorical variables) and perform Lasso and Ridge regression for both! Each time, look at R-squared and MSE.

## Lasso

With default parameter (alpha = 1)

In [14]:
# Your code here
from sklearn.linear_model import Lasso, Ridge

lasso = Lasso() 
lasso.fit(X_train_all, y_train_all)
print('Training r^2:', lasso.score(X_train_all, y_train_all))
print('Testing r^2:', lasso.score(X_test_all, y_test_all))
print('Training MSE:', mean_squared_error(y_train_all, lasso.predict(X_train_all)))
print('Testing MSE:', mean_squared_error(y_test_all, lasso.predict(X_test_all)))

Training r^2: 0.9391884242470414
Testing r^2: 0.8346885265417694
Training MSE: 382909263.3188206
Testing MSE: 1047468831.9226593


With a higher regularization parameter (alpha = 10)

In [15]:
# Your code here
lasso = Lasso(alpha=10) 
lasso.fit(X_train_all, y_train_all)
print('Training r^2:', lasso.score(X_train_all, y_train_all))
print('Testing r^2:', lasso.score(X_test_all, y_test_all))
print('Training MSE:', mean_squared_error(y_train_all, lasso.predict(X_train_all)))
print('Testing MSE:', mean_squared_error(y_test_all, lasso.predict(X_test_all)))

Training r^2: 0.9375066354449642
Testing r^2: 0.851942968030923
Training MSE: 393498900.2964467
Testing MSE: 938138914.9239562


## Ridge

With default parameter (alpha = 1)

In [16]:
# Your code here
ridge = Ridge(alpha=1)
ridge.fit(X_train, y_train)
print('Training r^2:', ridge.score(X_train, y_train))
print('Testing r^2:', ridge.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, ridge.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, ridge.predict(X_test)))

Training r^2: 0.8031754329398771
Testing r^2: 0.837923757960925
Training MSE: 1261380041.857263
Testing MSE: 969786955.3176246


With default parameter (alpha = 10)

In [17]:
# Your code here
ridge = Ridge(alpha=10) 
ridge.fit(X_train, y_train)
print('Training r^2:', ridge.score(X_train, y_train))
print('Testing r^2:', ridge.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, ridge.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, ridge.predict(X_test)))

Training r^2: 0.8031406707011477
Testing r^2: 0.8384070542246017
Training MSE: 1261602820.9279778
Testing MSE: 966895140.9087342


## Look at the metrics, what are your main conclusions?

Conclusions here

## Compare number of parameter estimates that are (very close to) 0 for Ridge and Lasso

In [20]:
# number of Ridge params almost zero
print(sum(abs(ridge.coef_) < 10**(-10)))

0


In [21]:
# number of Lasso params almost zero
print(sum(abs(lasso.coef_) < 10**(-10)))

70


Compare with the total length of the parameter space and draw conclusions!

In [22]:
# your code here
len(lasso.coef_)

289

## Summary

Great! You now know how to perform Lasso and Ridge regression.